In [7]:
import os
from os import listdir
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import cv2
from matplotlib.image import imread

import tensorflow as tf
from tensorflow.keras.utils import to_categorical  # Updated import
from tensorflow.keras.preprocessing import image  # Updated import
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from tqdm import tqdm

import glob
import PIL
import random

random.seed(100)

In [57]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("awsaf49/cbis-ddsm-breast-cancer-image-dataset")

print("Path to dataset files:", path)

100%|██████████| 4.95G/4.95G [02:05<00:00, 42.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset/versions/1


In [9]:
dicom_data = pd.read_csv('/content/dicom_info.csv')
calc_case_train_df = pd.read_csv('/content/calc_case_description_train_set.csv')
calc_case_test_df = pd.read_csv('/content/calc_case_description_test_set.csv')
mass_case_train_df = pd.read_csv('/content/mass_case_description_train_set.csv')
mass_case_test_df = pd.read_csv('/content/mass_case_description_test_set.csv')
image_dir = '/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/jpeg'

In [10]:
full_image_df = dicom_data[dicom_data['SeriesDescription'] == 'full mammogram images']
full_image_df['image_path'] = full_image_df['image_path'].apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))

<ipython-input-10-8a2cd857b93b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_image_df['image_path'] = full_image_df['image_path'].apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))


In [11]:
full_image_df.head(1)

,file_path,image_path,AccessionNumber,BitsAllocated,BitsStored,BodyPartExamined,Columns,ContentDate,ContentTime,ConversionType,...,SecondaryCaptureDeviceManufacturerModelName,SeriesDescription,SeriesInstanceUID,SeriesNumber,SmallestImagePixelValue,SpecificCharacterSet,StudyDate,StudyID,StudyInstanceUID,StudyTime
1,CBIS-DDSM/dicom/1.3.6.1.4.1.9590.100.1.2.24838...,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,NaN,16,16,BREAST,3526,20160426,143829.101,WSD,...,MATLAB,full mammogram images,1.3.6.1.4.1.9590.100.1.2.248386742010678582309...,1,0,ISO_IR 100,20160720.0,DDSM,1.3.6.1.4.1.9590.100.1.2.161516517311681906612...,193426.0


In [12]:
calc_case_train_df['SeriesInstanceUID'] = calc_case_train_df['image file path'].apply(lambda path: path.split('/')[2])
mass_case_train_df['SeriesInstanceUID'] = mass_case_train_df['image file path'].apply(lambda path: path.split('/')[2])

In [13]:
map_image_path_series_id = dict(zip(full_image_df['SeriesInstanceUID'].values,full_image_df['image_path'].values))

In [14]:
calc_case_train_df['image_path'] = calc_case_train_df['SeriesInstanceUID'].apply(lambda seri: map_image_path_series_id[seri])

In [15]:
calc_case_train_df.head(1)

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path,SeriesInstanceUID,image_path
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_CC/1.3.6.1.4.1.959...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...,1.3.6.1.4.1.9590.100.1.2.474143160103683865197...,/kaggle/input/cbis-ddsm-breast-cancer-image-da...


In [16]:
mass_case_train_df['image_path'] = mass_case_train_df['SeriesInstanceUID'].apply(lambda seri: map_image_path_series_id[seri])
mass_case_train_df.head(1)

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path,SeriesInstanceUID,image_path
0,P_00001,3,LEFT,CC,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,1.3.6.1.4.1.9590.100.1.2.342386194811267636608...,/kaggle/input/cbis-ddsm-breast-cancer-image-da...


In [17]:
calc_loc_df = calc_case_train_df[['patient_id', 'image_path', 'assessment', 'left or right breast', 'image view']]
calc_loc_df['id_side'] = calc_loc_df.apply(
    lambda row: f"{row['patient_id']}_{row['left or right breast']}", axis=1
)
print('calc_case_df sau khi lọc: ')
calc_loc_df

calc_case_df sau khi lọc: 


<ipython-input-17-e9ab61f5587a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calc_loc_df['id_side'] = calc_loc_df.apply(


,patient_id,image_path,assessment,left or right breast,image view,id_side
0,P_00005,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,CC,P_00005_RIGHT
1,P_00005,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,MLO,P_00005_RIGHT
2,P_00007,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,LEFT,CC,P_00007_LEFT
3,P_00007,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,LEFT,MLO,P_00007_LEFT
4,P_00008,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,2,LEFT,CC,P_00008_LEFT
...,...,...,...,...,...,...
1541,P_02566,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,5,RIGHT,MLO,P_02566_RIGHT
1542,P_02572,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,0,LEFT,CC,P_02572_LEFT
1543,P_02572,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,0,LEFT,MLO,P_02572_LEFT
1544,P_02584,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,0,LEFT,CC,P_02584_LEFT


from matplotlib import pyplot as plt
calc_loc_df['assessment'].plot(kind='hist', bins=20, title='assessment')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
calc_loc_df.groupby('left or right breast').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['image view'].value_counts()
    for x_label, grp in calc_loc_df.groupby('left or right breast')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('left or right breast')
_ = plt.ylabel('image view')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(calc_loc_df['left or right breast'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(calc_loc_df, x='assessment', y='left or right breast', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(calc_loc_df['image view'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(calc_loc_df, x='assessment', y='image view', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [18]:
mass_loc_df = mass_case_train_df[['patient_id', 'image_path', 'assessment', 'left or right breast', 'image view']]
mass_loc_df['id_side'] = mass_loc_df.apply(
    lambda row: f"{row['patient_id']}_{row['left or right breast']}", axis=1
)
mass_loc_df = mass_loc_df.sort_values(by=['patient_id', 'left or right breast'])
print('mass_case_df sau khi lọc và sắp xếp theo id và side: ')
mass_loc_df

mass_case_df sau khi lọc và sắp xếp theo id và side: 


<ipython-input-18-9772ae6ee08b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mass_loc_df['id_side'] = mass_loc_df.apply(


,patient_id,image_path,assessment,left or right breast,image view,id_side
0,P_00001,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,LEFT,CC,P_00001_LEFT
1,P_00001,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,LEFT,MLO,P_00001_LEFT
2,P_00004,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,LEFT,CC,P_00004_LEFT
3,P_00004,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,LEFT,MLO,P_00004_LEFT
4,P_00004,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,RIGHT,MLO,P_00004_RIGHT
...,...,...,...,...,...,...
1313,P_02033,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,MLO,P_02033_RIGHT
1314,P_02079,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,CC,P_02079_RIGHT
1315,P_02079,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,MLO,P_02079_RIGHT
1316,P_02092,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,LEFT,CC,P_02092_LEFT


from matplotlib import pyplot as plt
mass_loc_df['assessment'].plot(kind='hist', bins=20, title='assessment')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
mass_loc_df.groupby('left or right breast').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['image view'].value_counts()
    for x_label, grp in mass_loc_df.groupby('left or right breast')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('left or right breast')
_ = plt.ylabel('image view')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(mass_loc_df['left or right breast'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(mass_loc_df, x='assessment', y='left or right breast', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(mass_loc_df['image view'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(mass_loc_df, x='assessment', y='image view', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [19]:
train_df = pd.concat([calc_loc_df,mass_loc_df])
train_df

,patient_id,image_path,assessment,left or right breast,image view,id_side
0,P_00005,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,CC,P_00005_RIGHT
1,P_00005,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,MLO,P_00005_RIGHT
2,P_00007,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,LEFT,CC,P_00007_LEFT
3,P_00007,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,LEFT,MLO,P_00007_LEFT
4,P_00008,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,2,LEFT,CC,P_00008_LEFT
...,...,...,...,...,...,...
1313,P_02033,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,MLO,P_02033_RIGHT
1314,P_02079,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,CC,P_02079_RIGHT
1315,P_02079,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,MLO,P_02079_RIGHT
1316,P_02092,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,LEFT,CC,P_02092_LEFT


In [20]:
grouped = train_df.groupby('id_side').size()
grouped

,0
id_side,
P_00001_LEFT,2
P_00004_LEFT,2
P_00004_RIGHT,1
P_00005_RIGHT,2
P_00007_LEFT,2
...,...
P_02559_RIGHT,2
P_02563_RIGHT,2
P_02566_RIGHT,2


In [21]:
grouped = train_df.groupby('id_side').filter(lambda x: len(x) > 1)

# Giữ lại các nhóm mà cột 'image view' chứa cả 'CC' và 'MLO'
filtered_grouped = grouped.groupby('id_side').filter(
    lambda x: set(['CC', 'MLO']).issubset(set(x['image view']))
)

# Kết quả: DataFrame train_df chỉ còn các hàng thỏa điều kiện
train_df = train_df[train_df['id_side'].isin(filtered_grouped['id_side'])]
train_df

,patient_id,image_path,assessment,left or right breast,image view,id_side
0,P_00005,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,CC,P_00005_RIGHT
1,P_00005,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,MLO,P_00005_RIGHT
2,P_00007,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,LEFT,CC,P_00007_LEFT
3,P_00007,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,4,LEFT,MLO,P_00007_LEFT
4,P_00008,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,2,LEFT,CC,P_00008_LEFT
...,...,...,...,...,...,...
1313,P_02033,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,MLO,P_02033_RIGHT
1314,P_02079,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,CC,P_02079_RIGHT
1315,P_02079,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,RIGHT,MLO,P_02079_RIGHT
1316,P_02092,/kaggle/input/cbis-ddsm-breast-cancer-image-da...,3,LEFT,CC,P_02092_LEFT


In [22]:
class BreastDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
        self.id_side = dataframe['id_side'].unique()

    def __len__(self): #Train bao nhiêu sample = patientid_view
        return len(self.id_side)

    def __getitem__(self, idx):
        id_side = self.id_side[idx]
        sub = self.df[self.df['id_side']==id_side]
        try:
            path_cc = sub[sub['image view']=='CC']['image_path'].values[0] #String
            path_mlo = sub[sub['image view']=='MLO']['image_path'].values[0]
            label = sub['assessment'].max()
            image_cc = cv2.imread(path_cc)
            image_mlo = cv2.imread(path_mlo)
            image_cc = cv2.cvtColor(image_cc,cv2.COLOR_BGR2RGB)
            image_mlo = cv2.cvtColor(image_mlo,cv2.COLOR_BGR2RGB)
        except:
            print(id_side)
        if self.transform:
            image_cc = self.transform(image=image_cc)['image']
            image_mlo = self.transform(image=image_mlo)['image']
        return image_cc,image_mlo, int(label)

In [23]:
def get_augmentation(phase,input_size):
    if phase == 'train':
        return A.Compose([
               A.Resize(256, 256),
               A.Normalize(),
               ToTensorV2()
            ])
    elif phase in ['test','valid']:
        return A.Compose([
               A.Resize(256, 256),
               A.Normalize(),
               ToTensorV2()
        ])

In [24]:
train_dataset = BreastDataset(train_df, transform=get_augmentation('train',256))
#test_dataset = TumorDataset(test_df, transform=get_augmentation('test',256))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [25]:
# Mixup
def mixup_data(x1, x2, y, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    batch_size = x1.size(0)
    index = torch.randperm(batch_size)
    mixed_x1 = lam * x1 + (1 - lam) * x1[index, :]
    mixed_x2 = lam * x2 + (1 - lam) * x2[index, :]
    y_a, y_b = y, y[index]
    return mixed_x1, mixed_x2, y_a, y_b, lam


# CutMix
def cutmix_data(x1, x2, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    batch_size = x1.size(0)
    index = torch.randperm(batch_size)
    bbx1, bby1, bbx2, bby2 = rand_bbox(x1.size(), lam)
    x1[:, :, bbx1:bbx2, bby1:bby2] = x1[index, :, bbx1:bbx2, bby1:bby2]
    x2[:, :, bbx1:bbx2, bby1:bby2] = x2[index, :, bbx1:bbx2, bby1:bby2]
    y_a, y_b = y, y[index]
    return x1, x2, y_a, y_b, lam


def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

In [26]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels)
        )

        # Adjust skip connection if in_channels != out_channels
        self.skip_connection = nn.Sequential()
        if in_channels != out_channels:
            self.skip_connection = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        residual = self.skip_connection(x)
        x = self.conv_block(x)
        x += residual  # Adding the residual (skip connection)
        x = F.relu(x)
        return x

In [27]:
class CNNwithResidualBlocks(nn.Module):
    def __init__(self):
        super(CNNwithResidualBlocks, self).__init__()

        # Residual blocks
        self.block1 = ResidualBlock(3, 64)
        self.block2 = ResidualBlock(64, 32)
        self.block3 = ResidualBlock(32, 16)
        self.block4 = ResidualBlock(16, 8)

        # MaxPooling and Dropout
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.dropout = nn.Dropout(0.1)

        # Flatten Layer
        self.flatten = nn.Flatten()

        # After 4 pooling layers (each reducing size by half), size becomes 5x5 for each channel
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout5 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(2048, 6)

    def forward_feature(self, x):
    # Debug: in kích thước đầu vào

        x = self.block1(x)
        x = self.pool(x)
        x = self.dropout(x)

        x = self.block2(x)
        x = self.pool(x)
        x = self.dropout(x)

        x = self.block3(x)
        x = self.pool(x)
        x = self.dropout(x)

        x = self.block4(x)
        x = self.pool(x)
        x = self.dropout(x)

        x = self.flatten(x)
        x = F.relu(self.fc1(x))  # Feature map
        return x


    def forward(self, image_cc, image_mlo): #CC / MLO
        x1 = self.forward_feature(image_cc)
        x2 = self.forward_feature(image_mlo)
        x = torch.cat([x1, x2], dim=-1)  # Ghép chiều thứ 1 (feature)
        # Dense Layers
        x = self.dropout5(x)
        x = self.fc2(x)
        return x

In [28]:
model = CNNwithResidualBlocks().to('cuda')
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.to('cuda')
    for epoch in range(num_epochs):
        print(epoch)
        model.train()
        epoch_loss = 0.0
        for image_cc, image_mlo, labels in tqdm(train_loader):
            optimizer.zero_grad()
            labels = labels.to('cuda')
            image_cc = image_cc.to('cuda')
            image_mlo = image_mlo.to('cuda')

            #1 View: outputs = model(images)
            # Multiview:
            outputs = model(image_cc,image_mlo)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()  # Tính tổng loss trong epoch

        # In ra loss trung bình cho epoch
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(train_loader)}")